In [1]:
import torch
import torchvision.transforms as transforms
import torch.nn as nn
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import time

from resnet import ResNet50

In [ ]:
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    transforms.ToTensor()
])

val_transforms = transforms.Compose(
    [
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        transforms.ToTensor(),
    ]
)

In [ ]:
train = datasets.imagenette(root="/data", download=True, split="train", transforms=train_transforms)
train_loader = DataLoader(train, batch_size=4, shuffle=True)

test = datasets.imagenette(
    root="/data", download=True, split="val", transforms=test_transforms
)
test_loader = DataLoader(test, batch_size=4, shuffle=False)

In [ ]:
classes = [
    "tench", "English springer", "cassette player", "chain saw", "church", "French horn", "garbage truck", "gas pump", "golf ball", "parachute"
]

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = ResNet50(len(classes))
model.to(device)

for p in model.parameters():
    print(p)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, weight_decay=0.0001, momentum=0.9)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.1, patience=10)

In [ ]:
epochs = 50


losses = []
for epoch in range(epochs):
    start_time = time.perf_counter()
    train_loss = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)

        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()

    print(f"Epoch: {epoch} | Loss: {train_loss} | Time: {time.perf_counter() - start_time}")
    losses.append(train_loss)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(losses)
plt.show()

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Accuracy: {100 * correct / total}")

In [ ]:
torch.save(model.state_dict(), "resnet50.pth")